In [7]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve

from plotly.offline import iplot


from __future__ import print_function, division

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting style defaults
import seaborn as sns; sns.set()

In [8]:
df_X_train = pd.read_csv('Xtr.csv', header=None, usecols=np.arange(3072))
df_X_test = pd.read_csv('Xte.csv', header=None, usecols=np.arange(3072))
df_y_train = pd.read_csv('Ytr.csv')

In [9]:
X_train = np.array(df_X_train)
X_test = np.array(df_X_test)
y_train = np.array(df_y_train['Prediction'])

In [11]:
!pip install cvxopt

  Using cached cvxopt-1.1.9.tar.gz
  Running setup.py bdist_wheel for cvxopt: started
  Running setup.py bdist_wheel for cvxopt: finished with status 'error'
  Complete output from command C:\Users\Mazzo\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Mazzo\\AppData\\Local\\Temp\\pip-build-zana0oyr\\cvxopt\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Mazzo\AppData\Local\Temp\tmpwugvd_zlpip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.6
  creating build\lib.win-amd64-3.6\cvxopt
  copying src\python\coneprog.py -> build\lib.win-amd64-3.6\cvxopt
  copying src\python\cvxprog.py -> build\lib.win-amd64-3.6\cvxopt
  copying src\python\info.py -> build\lib.win-amd64-3.6\cvxopt
  copying src\python\misc.py -> build\lib.win-amd64-3.6\cvxopt
  copying src\python\mo

  Failed building wheel for cvxopt
Command "C:\Users\Mazzo\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Mazzo\\AppData\\Local\\Temp\\pip-build-zana0oyr\\cvxopt\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Mazzo\AppData\Local\Temp\pip-cvl0dek8-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Mazzo\AppData\Local\Temp\pip-build-zana0oyr\cvxopt\


In [19]:
!python setup.py install

python: can't open file 'setup.py': [Errno 2] No such file or directory


In [20]:
!conda install -c conda.anaconda.org/omnia cvxopt

Fetching package metadata ...............



channel with url: https://conda.anaconda.org/conda.anaconda.org/omnia/noarch

It is possible you have given conda an invalid channel. Please double-check
your conda configuration using `conda config --show`.

If the requested url is in fact a valid conda channel, please request that the
channel administrator create `noarch/repodata.json` and associated
`noarch/repodata.json.bz2` files, even if `noarch/repodata.json` is empty.


PackageNotFoundError: Package not found: '' Package missing in current win-64 channels: 
  - cvxopt

You can search for packages on anaconda.org with

    anaconda search -t conda cvxopt




In [22]:
!conda install -c omnia cvxopt 

Fetching package metadata ...............
Solving package specifications: .




UnsatisfiableError: The following specifications were found to be in conflict:
  - cvxopt -> python 2.7*
  - python 3.6*
Use "conda info <package>" to see the dependencies for each package.




In [23]:
!conda info python

Fetching package metadata .............

python 2.6.8 5
--------------
file name   : python-2.6.8-5.tar.bz2
name        : python
version     : 2.6.8
build string: 5
build number: 5
channel     : defaults
size        : 18.1 MB
arch        : x86_64
date        : 2013-02-04
license     : PSF
md5         : 30aefaec490819d8c5e85e7e52ffb4f2
platform    : win32
track_features: vc9
url         : https://repo.continuum.io/pkgs/free/win-64/python-2.6.8-5.tar.bz2
dependencies:
    pip

python 2.6.8 6
--------------
file name   : python-2.6.8-6.tar.bz2
name        : python
version     : 2.6.8
build string: 6
build number: 6
channel     : defaults
size        : 18.1 MB
arch        : x86_64
date        : 2013-02-25
license     : PSF
md5         : e7cdd5e86087e768f2bce08ffe4c148c
platform    : win32
track_features: vc9
url         : https://repo.continuum.io/pkgs/free/win-64/python-2.6.8-6.tar.bz2
dependencies:
    pip

python 2.6.9 0
--------------
file name   : python-2.6.9-0.tar.bz2
name        : 